In [1]:
#load the csv to the notebook
import pandas as pd

In [43]:
rawInput=pd.read_csv('lasvegasStrip.csv',encoding="ISO-8859-1")
rawInput.head()

,Unnamed: 0,user,location,date,rating,title,comments
0,0,s6steve,"Sheffield, United Kingdom","July 9, 2018",20,Over priced over rated,Over rated new Las Vegas has taken away the sp...
1,0,sathyanupheap,unknown,"July 9, 2018",40,Plenty to See...,I rented a Mini Cooper convertible and it was ...
2,0,Blu H,unknown,"July 9, 2018",50,City of anything goes!,If you haven¡¯t been here yet you¡¯re probably...
3,0,anita a,"Los Angeles, California","July 9, 2018",30,Not like it use to be,Back in the day you could eat and have fun on ...
4,0,kristycullinan,"Philadelphia, Pennsylvania","July 9, 2018",30,Walking walking and more walking.,The strip is 5 miles long and pretty spaced ou...


## preprocess text

In [44]:
#import the nltk lib for preprocessing
import nltk
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer

In [45]:
nltk.download() #make sure the computer has all nltk corpus

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [46]:
import string #for getting punctuation
import gensim

In [47]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()##WordNetLemmatizer is a class

In [48]:
##a def to clean the text(typical way), very nice
def clean(doc):
    stop_free = " ".join([i for i in gensim.utils.simple_preprocess(doc,min_len=3) if i not in stop])
    ##simple_preprocess can lowercase words and tokenize them,you can set up min len of token too,here is 3, so word like AI will not be included
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())##这个用来找stem
    return normalized

In [49]:
doc_complete=rawInput['comments'][:1000]

In [50]:
doc_clean = [clean(doc).split() for doc in doc_complete]    
#doc_clean is a list中包list,每个item是一个doc的tokens

In [51]:
doc_clean[:2]

[['rated',
  'new',
  'la',
  'vega',
  'taken',
  'away',
  'spark',
  'original',
  'strip',
  'prepared',
  'pant',
  'drink',
  'bother',
  'night',
  'club',
  'guy',
  'pretty',
  'sexist',
  'priced',
  'staff',
  'everywhere',
  'arrogant',
  'care',
  'long'],
 ['rented',
  'mini',
  'cooper',
  'convertible',
  'nice',
  'cruise',
  'around',
  'strip',
  'light',
  'sight',
  'sound',
  'crowd',
  'city',
  'never',
  'sleep',
  'something',
  'everyone',
  'young',
  'old']]

## create gensim corpus dicts and doc_term_Matrix

In [52]:
# Importing Gensim
import gensim
from gensim import corpora

In [53]:
# Creating the term dictionary of our courpus, where every unique term（token) is assigned an index. 
dictionary = corpora.Dictionary(doc_clean)

In [54]:
print(len(dictionary))
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

4009
0 arrogant
1 away
2 bother
3 care
4 club
5 drink
6 everywhere
7 guy
8 la
9 long
10 new


In [55]:
#### filter_extremes可以把出现less than 15 documents的tokens (absolute number) 
#or more than 0.5 documents (fraction of total dictionary size, not absolute number). dictionary里有一半都是那个词删掉。
#after the above two steps, keep only the first 100000 most frequent tokens.
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000) #当文字很多的时候用这个,先不run

In [56]:
# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

In [57]:
doc_term_matrix[:2]###这个matrix是每个doc的list of (token_id, token_count) tuples

[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1)],
 [(11, 1),
  (12, 1),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 1)]]

In [58]:
print (dictionary[0])#dictionary 可以直接输入token的id得出token
dictionary[doc_term_matrix[0][0][0]]#找到第一个id为0的词

away


'away'

## tf-idf model LDA
create tf-idf scores, 这个score就是给每个token一个score，这score的好处是：当很多doc都有某个词比如city，那么city对选择topic意义并不大，当有些词是某个doc独有如accessible，那么这个词的意义还更强。。
https://rare-technologies.com/pivoted-document-length-normalisation/

In [70]:
from gensim import corpora, models
tfidf = models.TfidfModel(doc_term_matrix)
corpus_tfidf = tfidf[doc_term_matrix]

In [71]:
corpus_tfidf[0]##第一个doc里面词的score，可以看到这个score跟doc_term_matrix是不一样的,出现太多的词score小

[(0, 0.38787917167478897),
 (1, 0.38787917167478897),
 (2, 0.25087690611269653),
 (3, 0.31076583463199875),
 (4, 0.18846085265220172),
 (5, 0.29003997957600197),
 (6, 0.29486214935076166),
 (7, 0.14284428607598382),
 (8, 0.41908719840503633),
 (9, 0.3576440748534904),
 (10, 0.09514539559335049)]

In [75]:
lda_model_tfidf = gensim.models.ldamodel.LdaModel(corpus_tfidf, num_topics=10, id2word=dictionary, passes=50)

In [76]:
lda_model_tfidf.show_topics(num_topics=10, num_words=20)##可以发现tfidf去掉了很多general的词。

[(0,
  '0.027*"ride" + 0.024*"picture" + 0.023*"open" + 0.023*"eye" + 0.020*"costume" + 0.019*"dressed" + 0.018*"buy" + 0.018*"trying" + 0.017*"bus" + 0.016*"nice" + 0.015*"thought" + 0.014*"lot" + 0.014*"walk" + 0.013*"pas" + 0.013*"sell" + 0.013*"clean" + 0.013*"cheaper" + 0.012*"remember" + 0.012*"night" + 0.012*"walking"'),
 (1,
  '0.027*"always" + 0.027*"love" + 0.020*"something" + 0.018*"experience" + 0.018*"watching" + 0.018*"new" + 0.018*"different" + 0.016*"fun" + 0.016*"great" + 0.016*"people" + 0.015*"seeing" + 0.015*"time" + 0.015*"see" + 0.014*"recommend" + 0.014*"shopping" + 0.014*"year" + 0.014*"lot" + 0.013*"eating" + 0.013*"explore" + 0.013*"load"'),
 (2,
  '0.010*"get" + 0.010*"hotel" + 0.010*"time" + 0.009*"vega" + 0.009*"day" + 0.009*"walk" + 0.009*"one" + 0.009*"see" + 0.009*"casino" + 0.008*"walking" + 0.008*"night" + 0.008*"people" + 0.008*"like" + 0.008*"much" + 0.007*"place" + 0.007*"lot" + 0.007*"la" + 0.007*"many" + 0.007*"thing" + 0.007*"take"'),
 (3,
  '0.0

## typical LDA model(bag of words)

In [63]:
# Creating the object for LDA model using gensim library
lda_model = gensim.models.LdaMulticore(doc_term_matrix, num_topics=5, id2word=dictionary, passes=2, workers=2)
#这里workers是用几个cpu的意思，multicore就是多核运算意思

In [64]:
print(lda_model.print_topics(num_topics=5, num_words=10))

[(0, '0.046*"vega" + 0.024*"casino" + 0.023*"la" + 0.022*"people" + 0.018*"hotel" + 0.018*"lot" + 0.017*"fun" + 0.016*"one" + 0.016*"place" + 0.016*"great"'), (1, '0.036*"people" + 0.031*"night" + 0.029*"see" + 0.022*"walk" + 0.021*"light" + 0.020*"walking" + 0.018*"get" + 0.017*"lot" + 0.016*"casino" + 0.014*"day"'), (2, '0.034*"vega" + 0.024*"hotel" + 0.016*"walking" + 0.016*"like" + 0.014*"place" + 0.013*"day" + 0.013*"la" + 0.012*"lot" + 0.012*"see" + 0.012*"one"'), (3, '0.036*"see" + 0.025*"place" + 0.024*"hotel" + 0.022*"walk" + 0.019*"time" + 0.019*"vega" + 0.019*"much" + 0.016*"casino" + 0.013*"show" + 0.013*"walking"'), (4, '0.044*"vega" + 0.032*"time" + 0.030*"people" + 0.020*"day" + 0.018*"hotel" + 0.017*"show" + 0.015*"la" + 0.015*"take" + 0.014*"like" + 0.014*"walking"')]


In [65]:
lda_model.show_topics(num_topics=5, num_words=10)

[(0,
  '0.046*"vega" + 0.024*"casino" + 0.023*"la" + 0.022*"people" + 0.018*"hotel" + 0.018*"lot" + 0.017*"fun" + 0.016*"one" + 0.016*"place" + 0.016*"great"'),
 (1,
  '0.036*"people" + 0.031*"night" + 0.029*"see" + 0.022*"walk" + 0.021*"light" + 0.020*"walking" + 0.018*"get" + 0.017*"lot" + 0.016*"casino" + 0.014*"day"'),
 (2,
  '0.034*"vega" + 0.024*"hotel" + 0.016*"walking" + 0.016*"like" + 0.014*"place" + 0.013*"day" + 0.013*"la" + 0.012*"lot" + 0.012*"see" + 0.012*"one"'),
 (3,
  '0.036*"see" + 0.025*"place" + 0.024*"hotel" + 0.022*"walk" + 0.019*"time" + 0.019*"vega" + 0.019*"much" + 0.016*"casino" + 0.013*"show" + 0.013*"walking"'),
 (4,
  '0.044*"vega" + 0.032*"time" + 0.030*"people" + 0.020*"day" + 0.018*"hotel" + 0.017*"show" + 0.015*"la" + 0.015*"take" + 0.014*"like" + 0.014*"walking"')]

## pyLDAvis  
dictionary/lda/doc_term_matrix可以先保存,然后load的

In [66]:
"""
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
doc_term_matrix = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model5.gensim')
"""

"\ndictionary = gensim.corpora.Dictionary.load('dictionary.gensim')\ndoc_term_matrix = pickle.load(open('corpus.pkl', 'rb'))\nlda = gensim.models.ldamodel.LdaModel.load('model5.gensim')\n"

In [77]:
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda_model_tfidf, doc_term_matrix, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)